<a href="https://colab.research.google.com/github/jfogarty/machine-learning-intro-workshop/blob/master/notebooks/transfer_learning_keras_MobileNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
URL='https://github.com/aditya9898/transfer-learning/tree/master/train'

## Copy the Transfer Learning Training image to here.

This uses some nasty tricks to get around githubusercontent.com's refusal to allow wget downloading of entire directories.  This is really nasty.

In [0]:
import numpy as np
import requests
import shutil
import os

ds = np.DataSource()
def copyHere(URL, toPath, quiet=False):
    toDir, toFile = os.path.split(toPath)
    toPath = os.path.join(toDir, toFile)
    if os.path.exists(toPath):
        if not quiet:
            print(f"- Skipped copy of existing file {toPath}.")
    else:
        if ds.exists(URL):
            if not toFile:
                urlPrefix, toFile = os.path.split(URL)
            response = requests.get(URL, stream=True)
            if toDir:
                if not os.path.exists(toDir): 
                  print(f"- Creating directory '{toDir}'.")
                  os.makedirs(toDir)
            with open(toPath, 'wb') as fin: shutil.copyfileobj(response.raw, fin)
            if not quiet:
                print(f"- Copied {URL} to {toPath}.")
        else:
            print(f"** Sorry, can't copy '{URL}' to '{toPath}'.")

In [0]:
import requests
from bs4 import BeautifulSoup

def get_url_files(url, ext='', params={}):
    response = requests.get(url, params=params)
    if response.ok:
        response_text = response.text
    else:
        return response.raise_for_status()
    soup = BeautifulSoup(response_text, 'html.parser')
    parent = [url + node.get('href') for node in soup.find_all('a') if node.get('href').endswith(ext)]
    files = [os.path.basename(f) for f in parent]    
    return files

In [17]:
filename = 'Z.jpg'
REPO     = 'https://raw.githubusercontent.com/aditya9898/transfer-learning/'
BRANCH   = 'master/'
DIR      = 'train/cats/'
TMPDIR   = 'tmpData'

url = 'https://github.com/aditya9898/transfer-learning/tree/master/'
ext = 'jpg'

for imgdir in ['train/cats', 'train/dogs', 'train/horses']:
    TODIR = os.path.join(TMPDIR, imgdir)
    files =  get_url_files(url + imgdir, ext)
    print(f"- Downloading all images to {TODIR}:")
    for img in files:
        URL = os.path.join(REPO, BRANCH, imgdir, img)
        toPath  = os.path.join(TODIR, img)
        #print(f'- Copy {URL} to {toPath}')
        copyHere(URL, toPath, quiet=True)

- Downloading all images to tmpData/train/cats:
- Downloading all images to tmpData/train/dogs:
- Downloading all images to tmpData/train/horses:


In [0]:

url = 'https://raw.githubusercontent.com/aditya9898/transfer-learning/master/train/cats'
url = 'https://github.com/aditya9898/transfer-learning/tree/master/train/cats'
ext = 'jpg'
result = get_url_paths(url, ext)

print(result)
files = [os.path.basename(f) for f in result]
print(files)

In [27]:
!ls -al

total 24
drwxr-xr-x 1 root root 4096 Aug 15 18:07 .
drwxr-xr-x 1 root root 4096 Aug 15 15:37 ..
drwxr-xr-x 1 root root 4096 Aug 13 16:04 .config
drwxr-xr-x 1 root root 4096 Aug  2 16:06 sample_data
drwxr-xr-x 3 root root 4096 Aug 15 17:48 tmpData
-rw-r--r-- 1 root root   15 Aug 15 18:07 xxx


In [1]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

Using TensorFlow backend.


## xxx

In [2]:
base_model=MobileNet(weights='imagenet', include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(3,activation='softmax')(x) #final layer with softmax activation

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '
W0815 17:28:37.043514 140413328377728 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0815 17:28:37.093657 140413328377728 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0815 17:28:37.117455 140413328377728 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.un

17227776/17225924 [==============================] - 1s 0us/step


## xxx

In [0]:
model=Model(inputs=base_model.input,outputs=preds)
#specify the inputs
#specify the outputs
#now a model has been created based on our architecture

## xxx

In [0]:
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

## xxx

In [0]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)

In [0]:
 #included in our dependencies
# ./train/ has been downloaded from https://github.com/aditya9898/transfer-learning/tree/master/train
train_generator=train_datagen.flow_from_directory('./train/', 
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)